# Collate Collections of Wet Depths by Month for Future Conditions: CN1, Custom Climate Normal 2031-2060, and CN2

The purpose of this notebook is to collate collections of wet day depths by month.

## Imports and Parameters

In [1]:
%matplotlib inline

In [2]:
import os
import pickle
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import datetime as dt
from copy import deepcopy
from IPython.display import display, HTML, Image

In [3]:
IN_DIR1 = r'C:\Users\nmartin\Documents\EAA_HCP\Data\SwRI_Processed\CMIP6'
IN_DIR2 = r'C:\Users\nmartin\Documents\EAA_HCP\Data\SwRI_Processed\CMIP5'
IN_DIR3 = r'C:\Users\nmartin\Documents\EAA_HCP\Data\SwRI_Processed\DayMet_GridIntersect'

In [4]:
OUT_DIR = r'C:\Users\nmartin\Documents\EAA_HCP\Data\SwRI_Processed\CombinedFuture'

In [5]:
# load the regular day indexes
DAY_INDEX_FILE = "Future_RegYr_Index_DF.pkl"
InFiler = os.path.normpath( os.path.join( IN_DIR3, DAY_INDEX_FILE ) )
DAYIND_DF_C6 = pd.read_pickle( InFiler )
START_2021_C6 = 2190
START_2051_C6 = 13140
START_2081_C6 = 24090
DAY_INDEX_FILE = "CMIP5Future_RegYr_Index_DF.pkl"
InFiler = os.path.normpath( os.path.join( IN_DIR3, DAY_INDEX_FILE ) )
DAYIND_DF_C5 = pd.read_pickle( InFiler )
START_2021_C5 = 5475
START_2051_C5 = 16425
START_2081_C5 = 27375

In [6]:
CCNStartTS = pd.Timestamp( 2031, 1, 1, )
CCNStartYr = CCNStartTS.year
CCNStopTS = pd.Timestamp( 2060, 12, 31, 23, 59, )
CCNStopYr = CCNStopTS.year

In [7]:
MODEL_DICT = { 2 : ["CMIP6", ["ssp245", "ssp585"], ], 
               3 : ["CMIP6", ["ssp245", "ssp585"], ], 
               4 : ["CMIP6", ["ssp245", "ssp585"], ], 
               5 : ["CMIP6", ["ssp245", "ssp585"], ], 
               11 : ["CMIP6", ["ssp245", "ssp585"], ], 
               12 : ["CMIP6", ["ssp245", "ssp585"], ], 
               7 : ["CMIP5", ["rcp45", "rcp85"], ], 
               8 : ["CMIP5", ["rcp45", "rcp85"], ], 
               9 : ["CMIP5", ["rcp45", "rcp85"], ], 
               10 : ["CMIP5", ["rcp85"], ], }
MODEL_KEYS = list( MODEL_DICT.keys() )

In [8]:
# full basin intersection dictionary
InFiler = os.path.normpath( os.path.join( IN_DIR3, "BasWeightsGDF.pkl" ) )
with open( InFiler, 'rb' ) as IF:
    BasWeightsDF = pickle.load( IF )
# end with
BAS_KEYS = sorted( BasWeightsDF.keys() )

In [9]:
# cutoff threshold depth for precipitation in millimeters
#    equivalent to 0.01 inch
CutThresh = 0.255

## Custom CN, 2031-2060

### Make Storage Dictionaries

For these values will store as lists in a dictionary with keys that are integer months.

In [10]:
FB_Wet_Depth = dict()

In [11]:
for bas in BAS_KEYS:
    localDict = dict()
    for mM in range(1, 13, 1):
        localDict[mM] = list()
    # end for
    FB_Wet_Depth[bas] = deepcopy( localDict )
# end for

### Collate Basins

Loop through the keys and do one basin at a time. Iterate through the DataFrame and allocate contiguous counts to month

In [12]:
for bas in BAS_KEYS:
    for mKey in MODEL_KEYS:
        modelMeta = MODEL_DICT[mKey]
        scenList = modelMeta[1]
        if modelMeta[0] == "CMIP6":
            inDir = IN_DIR1
            DayIndDF1 = DAYIND_DF_C6.loc[START_2021_C6:(START_2051_C6-1)].copy()
            DayIndDF2 = DAYIND_DF_C6.loc[START_2051_C6:(START_2081_C6-1)].copy()
        elif modelMeta[0] == "CMIP5":
            inDir = IN_DIR2
            DayIndDF1 = DAYIND_DF_C5.loc[START_2021_C5:(START_2051_C5-1)].copy()
            DayIndDF2 = DAYIND_DF_C5.loc[START_2051_C5:(START_2081_C5-1)].copy()
        else:
            errMsg = "Invalid CMIP of %s!!!" % modelMeta[0]
            print("%s" % errMsg)
        for tSen in scenList:
            iFileName = "Mod_%02d_%s-CN1_2021to2050_%s_DF.pkl" % ( mKey, tSen, bas )
            InFiler = os.path.normpath( os.path.join( inDir, iFileName ) )
            cBasDF1 = pd.read_pickle( InFiler )
            cBasDF1 = DayIndDF1.merge( cBasDF1, how='inner', left_index=True, right_index=True )
            iFileName = "Mod_%02d_%s-CN2_2051to2080_%s_DF.pkl" % ( mKey, tSen, bas )
            InFiler = os.path.normpath( os.path.join( inDir, iFileName ) )
            cBasDF2 = pd.read_pickle( InFiler )
            cBasDF2 = DayIndDF2.merge( cBasDF2, how='inner', left_index=True, right_index=True )
            cBasDF = pd.concat( [cBasDF1, cBasDF2], )
            cMetDF = cBasDF[(cBasDF["Year"] >= CCNStartYr) & (cBasDF["Year"] <= CCNStopYr)].copy()
            if len(cMetDF) != (30*365):
                # this is an error
                print("Custom CN met has length %d and %d was expected!!!" % (len(cMetDF),(30*365)))
            # end if
            # start calculations for this model
            oCnt = 0
            for indx, row in cMetDF.iterrows():
                curMonth = int( row["Month"] )
                curPre = float( row["Pre_mm"] )
                if curPre >= CutThresh:
                    # then current day is wet
                    FB_Wet_Depth[bas][curMonth].append( curPre )
                # increment the overall counter
                oCnt += 1
            # end of DataFrame iteration for
        # end of scenario for
    # end of model for
# end of basin iteration for

Now serialize for future use

In [13]:
OutFiler = os.path.normpath( os.path.join( OUT_DIR, "CCN_FullB_WetDepth_2031through2060_MonDict.pkl" ) )
with open( OutFiler, 'wb' ) as OF:
    pickle.dump( FB_Wet_Depth, OF, protocol=pickle.HIGHEST_PROTOCOL )
# end with

## CN1, 2021-2050

### Make Storage Dictionaries

For these values will store as lists in a dictionary with keys that are integer months.

In [14]:
FB_Wet_Depth = dict()

In [15]:
for bas in BAS_KEYS:
    localDict = dict()
    for mM in range(1, 13, 1):
        localDict[mM] = list()
    # end for
    FB_Wet_Depth[bas] = deepcopy( localDict )
# end for

### Collate Basins

Loop through the keys and do one basin at a time. Iterate through the DataFrame and allocate contiguous counts to month

In [16]:
for bas in BAS_KEYS:
    for mKey in MODEL_KEYS:
        modelMeta = MODEL_DICT[mKey]
        scenList = modelMeta[1]
        if modelMeta[0] == "CMIP6":
            inDir = IN_DIR1
            DayIndDF = DAYIND_DF_C6.loc[START_2021_C6:(START_2051_C6-1)].copy()
        elif modelMeta[0] == "CMIP5":
            inDir = IN_DIR2
            DayIndDF = DAYIND_DF_C5.loc[START_2021_C5:(START_2051_C5-1)].copy()
        else:
            errMsg = "Invalid CMIP of %s!!!" % modelMeta[0]
            print("%s" % errMsg)
        # end if
        for tSen in scenList:
            iFileName = "Mod_%02d_%s-CN1_2021to2050_%s_DF.pkl" % ( mKey, tSen, bas )
            InFiler = os.path.normpath( os.path.join( inDir, iFileName ) )
            cBasDF = pd.read_pickle( InFiler )
            cMetDF = DayIndDF.merge( cBasDF, how='inner', left_index=True,
                                     right_index=True )
            # start calculations for this model
            oCnt = 0
            for indx, row in cMetDF.iterrows():
                curMonth = int( row["Month"] )
                curPre = float( row["Pre_mm"] )
                if curPre >= CutThresh:
                    # then current day is wet
                    FB_Wet_Depth[bas][curMonth].append( curPre )
                # increment the overall counter
                oCnt += 1
            # end of DataFrame iteration for
        # end of scenario for
    # end of model for
# end of basin iteration for

Now serialize for future use

In [17]:
OutFiler = os.path.normpath( os.path.join( OUT_DIR, "CN1_FullB_WetDepth_2021through2050_MonDict.pkl" ) )
with open( OutFiler, 'wb' ) as OF:
    pickle.dump( FB_Wet_Depth, OF, protocol=pickle.HIGHEST_PROTOCOL )
# end with

## CN2, 2051-2080

### Make Storage Dictionaries

For these values will store as lists in a dictionary with keys that are integer months.

In [18]:
FB_Wet_Depth = dict()

In [19]:
for bas in BAS_KEYS:
    localDict = dict()
    for mM in range(1, 13, 1):
        localDict[mM] = list()
    # end for
    FB_Wet_Depth[bas] = deepcopy( localDict )
# end for

### Collate Basins

Loop through the keys and do one basin at a time. Iterate through the DataFrame and allocate contiguous counts to month

In [20]:
for bas in BAS_KEYS:
    for mKey in MODEL_KEYS:
        modelMeta = MODEL_DICT[mKey]
        scenList = modelMeta[1]
        if modelMeta[0] == "CMIP6":
            inDir = IN_DIR1
            DayIndDF = DAYIND_DF_C6.loc[START_2051_C6:(START_2081_C6-1)].copy()
        elif modelMeta[0] == "CMIP5":
            inDir = IN_DIR2
            DayIndDF = DAYIND_DF_C5.loc[START_2051_C5:(START_2081_C5-1)].copy()
        else:
            errMsg = "Invalid CMIP of %s!!!" % modelMeta[0]
            print("%s" % errMsg)
        # end if
        for tSen in scenList:
            iFileName = "Mod_%02d_%s-CN2_2051to2080_%s_DF.pkl" % ( mKey, tSen, bas )
            InFiler = os.path.normpath( os.path.join( inDir, iFileName ) )
            cBasDF = pd.read_pickle( InFiler )
            cMetDF = DayIndDF.merge( cBasDF, how='inner', left_index=True,
                                     right_index=True )
            # start calculations for this model
            oCnt = 0
            for indx, row in cMetDF.iterrows():
                curMonth = int( row["Month"] )
                curPre = float( row["Pre_mm"] )
                if curPre >= CutThresh:
                    # then current day is wet
                    FB_Wet_Depth[bas][curMonth].append( curPre )
                # increment the overall counter
                oCnt += 1
            # end of DataFrame iteration for
        # end of scenario for
    # end of model for
# end of basin iteration for

Now serialize for future use

In [21]:
OutFiler = os.path.normpath( os.path.join( OUT_DIR, "CN2_FullB_WetDepth_2051through2080_MonDict.pkl" ) )
with open( OutFiler, 'wb' ) as OF:
    pickle.dump( FB_Wet_Depth, OF, protocol=pickle.HIGHEST_PROTOCOL )
# end with